# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [183]:
# import libraries
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
import re
import sys
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_multilabel_classification
from sqlalchemy import create_engine
import nltk
import warnings
warnings.filterwarnings('ignore')

In [134]:
# load data from database
def load_data():
    '''
    Load data, transform DataFrame, get X, Y and name of feature columns for score results
    
    INPUT:
    engine - create data base
    df - read a table of engine
    
    OUTPUT:
    X - an array with columns messages from df
    Y - a new dataframe that has the following characteristics:
            1. has all comns except for 'id', 'message', 'original', and 'genre'.
            2. has a column 'related' cleaned from values 2.
    categiries - a list of columns names in Y.    
    '''
    
    engine = create_engine('sqlite:///messages.db')
    df = pd.read_sql_table("messages", engine)  
    X = df['message'].values
    Y = df.drop(['id', 'message', 'original', 'genre'],axis=1)
    Y['related'] = Y['related'].map(lambda x: 1 if x == 2 else x)
    categories = Y.columns
    
    return X, Y, categories

In [135]:
X, Y, categories = load_data()

In [246]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [136]:
categories

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [108]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    '''
    Clean, normalize, tokenize, lemmatize a text
    
    INPUT:
    text - a string, in this case messages
    
    OUTPUT:
    clean_tokens - an array with a text that has the following characteristics:
            1. has no punctuation marks
            2. splited into sequence of words
            3. cleaned from stopwords
            4. lemmatized
            5. all letters are in low case
    '''
    
    #normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize
    tokens = word_tokenize(text)
    
    #stop_words
    my_stopwords=stopwords.words('english')
    tokens = [word for word in tokens if word not in my_stopwords]
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
def simple_model():
    '''
    Create a pipeline for the Machine Learning
    
    OUTPUT:
    pipeline that has:
            1. transform estimator with tokenize function
            2. transform estimator TF IDF
            3. classifier multi output estimator
    '''
    
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ]) 
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [153]:
def main_simple():
      
    '''
    Main function that:
        1. loads datas and gets X, Y und categories
        2. splits the data into training and test 
        3. defines a model - simple_model
        4. fits the model
        5. predicts on test data
        6. prints results for each feature
    
    '''
        
    X, Y, categories = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    # train classifier
    model = simple_model()
    model.fit(X_train, y_train)
    
    # predict on test data
    y_pred = model.predict(X_test)
    
    #score report
    y_pred_pd = pd.DataFrame(y_pred, columns = categories)
    for column in categories:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [154]:
main_simple()

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1528
           1       0.86      0.91      0.88      5026

    accuracy                           0.82      6554
   macro avg       0.74      0.70      0.72      6554
weighted avg       0.80      0.82      0.81      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5450
           1       0.75      0.45      0.56      1104

    accuracy                           0.88      6554
   macro avg       0.83      0.71      0.75      6554
weighted avg       0.87      0.88      0.87      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1     

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6013
           1       0.86      0.24      0.38       541

    accuracy                           0.93      6554
   macro avg       0.90      0.62      0.67      6554
weighted avg       0.93      0.93      0.92      6554

------------------------------------------------------

FEATURE: storm

              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5966
           1       0.73      0.37      0.49       588

    accuracy                           0.93      6554
   macro avg       0.84      0.68      0.73      6554
weighted avg       0.92      0.93      0.92      6554

------------------------------------------------------

FEATURE: fire

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       1.00      0.01      0.02        80

    accuracy                           

### 6. Improve your model
Use grid search to find better parameters. 

In [155]:
def main_grid():
    
    '''
    Main function for grid search that:
        1. loads datas and gets X, Y und categories
        2. splits the data into training and test 
        3. defines a pipeline as a model - simple_model pipeline
        4. sets parameters for an estimator
        5. defines grid search model with the pipeline and parameters
        6. fits the model
        7. predicts on test data
        8. prints results for each feature
    
    '''
    X, Y, categories = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    model = simple_model()
    
    #use the same pipeline
    #add parameters
    parameters = {
        'clf__estimator__n_estimators': [20, 50]
    }
    
    cv = GridSearchCV(estimator=model, param_grid=parameters, cv=3, scoring='f1_weighted', verbose=3)
    
    #fitting
    cv.fit(X_train, y_train)
    
    #predict
    y_pred = cv.predict(X_test)
    
    #score report
    y_pred_pd = pd.DataFrame(y_pred, columns = categories)
    for column in categories:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [156]:
main_grid()

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... clf__estimator__n_estimators=20, score=0.539, total=  56.4s
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   56.5s remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=20, score=0.529, total= 1.0min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=20, score=0.544, total= 1.0min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.545, total= 2.2min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.539, total= 2.2min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.551, total= 2.2min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  9.6min finished


------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.70      0.45      0.55      1539
           1       0.85      0.94      0.89      5015

    accuracy                           0.83      6554
   macro avg       0.78      0.70      0.72      6554
weighted avg       0.81      0.83      0.81      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5433
           1       0.81      0.54      0.65      1121

    accuracy                           0.90      6554
   macro avg       0.86      0.76      0.80      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1     

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6419
           1       0.67      0.01      0.03       135

    accuracy                           0.98      6554
   macro avg       0.82      0.51      0.51      6554
weighted avg       0.97      0.98      0.97      6554

------------------------------------------------------

FEATURE: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6517
           1       0.00      0.00      0.00        37

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

------------------------------------------------------

FEATURE: hospitals

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.00      0.00      0.00        74

    accuracy                      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [240]:
def improved_model():
    
    '''
    Create a pipeline and parameters for a grid search model
    
    OUTPUT:
    cv - model that:
            1. defines an improved pipeline 
            2. sets parameters for estimators
            3. defins a grid search model with the pipeline and parameters
    '''
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
        ))
    ])

    parameters = {
        'clf__estimator__learning_rate': [0.1, 0.3],
        'clf__estimator__n_estimators': [100, 200]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [241]:
def main_improved():
    
    '''
    Main function for grid search that:
        1. loads datas and gets X, Y und categories
        2. splits the data into training and test 
        3. defines a model - improved_model
        4. fits the model
        5. predicts on test data
        6. prints results for each feature
    
    '''
    
    X, Y, categories = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    
  
    model = improved_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    #score report
    y_pred_pd = pd.DataFrame(y_pred, columns = categories)
    for column in categories:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

In [245]:
main_improved()

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.45      0.86      0.59      1522
           1       0.94      0.68      0.79      5032

    accuracy                           0.72      6554
   macro avg       0.70      0.77      0.69      6554
weighted avg       0.83      0.72      0.74      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.95      0.88      0.91      5443
           1       0.56      0.77      0.65      1111

    accuracy                           0.86      6554
   macro avg       0.76      0.82      0.78      6554
weighted avg       0.88      0.86      0.87      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      6525
           1     

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5955
           1       0.83      0.83      0.83       599

    accuracy                           0.97      6554
   macro avg       0.91      0.91      0.91      6554
weighted avg       0.97      0.97      0.97      6554

------------------------------------------------------

FEATURE: cold

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      6430
           1       0.34      0.65      0.44       124

    accuracy                           0.97      6554
   macro avg       0.66      0.81      0.71      6554
weighted avg       0.98      0.97      0.97      6554

------------------------------------------------------

FEATURE: other_weather

              precision    recall  f1-score   support

           0       0.97      0.88      0.92      6212
           1       0.20      0.56      0.30       342

    accuracy                   

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model, open('clasifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.